# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pathlib
from pprint import pprint
# Import API key
from api_keys import g_key

In [2]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Read CSV
csvpath = pathlib.Path("../output_data/cities.csv")
df = pd.read_csv(csvpath)
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.08,-159.32,73.40,73.0,40.0,13.87,US,1589126037
1,1,pevek,69.70,170.31,34.12,87.0,68.0,14.07,RU,1589126037
2,2,barrow,71.29,-156.79,21.20,92.0,1.0,5.82,US,1589126037
3,3,alyangula,-13.85,136.42,73.40,94.0,5.0,1.12,AU,1589126037
4,4,butaritari,3.07,172.79,82.62,78.0,18.0,7.99,KI,1589126038


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
location = df[["Lat", "Lng"]]
humidity = df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(
    location,
    weights = humidity,
    dissipating=False,
    max_intensity=100,
    point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = df.loc[df["Max Temp"] > 70].reset_index()
hotel_df = df.loc[df["Max Temp"] < 80]
hotel_df = df.loc[df["Cloudiness"] == 0]
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df[["City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hermanus,-34.42,19.23,69.01,70.0,0.0,0.92,ZA,1589126038,
1,beitbridge,-22.22,30.00,69.80,47.0,0.0,11.74,ZW,1589126040,
2,luderitz,-26.65,15.16,84.20,32.0,0.0,6.93,NaN,1589126044,
3,suluktu,39.93,69.57,71.60,53.0,0.0,2.24,KG,1589126045,
4,santa maria del oro,25.93,-105.37,73.89,27.0,0.0,5.75,MX,1589126047,
...,...,...,...,...,...,...,...,...,...,...
59,mathbaria,22.29,89.96,83.95,74.0,0.0,5.82,BD,1589126151,
60,pyapon,16.28,95.68,87.08,58.0,0.0,9.35,MM,1589126152,
61,buri,-23.80,-48.59,74.82,48.0,0.0,6.35,BR,1589126156,
62,jhol,25.96,68.89,98.60,47.0,0.0,11.41,PK,1589126157,


In [8]:
radius = 5000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
"types": target_type,
"radius": radius,
"key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    hotel_name = requests.get(base_url, params).json()
    if len(hotel_name["results"]) > 0:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    elif len(hotel_name["results"]) == 0:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hermanus,-34.42,19.23,69.01,70.0,0.0,0.92,ZA,1589126038,Misty Waves Boutique Hotel
1,beitbridge,-22.22,30.00,69.80,47.0,0.0,11.74,ZW,1589126040,PETERS MOTEL & CONFERENCE CENTRE
2,luderitz,-26.65,15.16,84.20,32.0,0.0,6.93,NaN,1589126044,LÜDERITZ NEST HOTEL
3,suluktu,39.93,69.57,71.60,53.0,0.0,2.24,KG,1589126045,NaN
4,santa maria del oro,25.93,-105.37,73.89,27.0,0.0,5.75,MX,1589126047,Hotel Palacio Del Sol
...,...,...,...,...,...,...,...,...,...,...
59,mathbaria,22.29,89.96,83.95,74.0,0.0,5.82,BD,1589126151,বাহাদুরের চায়ের ষ্টোর
60,pyapon,16.28,95.68,87.08,58.0,0.0,9.35,MM,1589126152,Aung Naing Thu Motel
61,buri,-23.80,-48.59,74.82,48.0,0.0,6.35,BR,1589126156,Hotel Aro
62,jhol,25.96,68.89,98.60,47.0,0.0,11.41,PK,1589126157,Vilage peroz khan thaheem


In [9]:
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "NaN"]
hotel_df.head(40)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hermanus,-34.42,19.23,69.01,70.0,0.0,0.92,ZA,1589126038,Misty Waves Boutique Hotel
1,beitbridge,-22.22,30.00,69.80,47.0,0.0,11.74,ZW,1589126040,PETERS MOTEL & CONFERENCE CENTRE
2,luderitz,-26.65,15.16,84.20,32.0,0.0,6.93,NaN,1589126044,LÜDERITZ NEST HOTEL
4,santa maria del oro,25.93,-105.37,73.89,27.0,0.0,5.75,MX,1589126047,Hotel Palacio Del Sol
5,port elizabeth,-33.92,25.57,75.20,41.0,0.0,9.17,ZA,1589126047,39 On Nile Guest House
6,rawson,-43.30,-65.10,62.08,35.0,0.0,27.20,AR,1589126049,Hotel Punta León
8,komsomolskiy,40.43,71.72,73.40,40.0,0.0,4.70,UZ,1589126052,"Hotel ""Ikathouse"""
9,cape town,-33.93,18.42,68.00,72.0,0.0,8.05,ZA,1589125889,Southern Sun Waterfront Cape Town
10,nyurba,63.28,118.33,32.88,92.0,0.0,5.50,RU,1589126055,Gostinitsa Alrosa
11,saldanha,-33.01,17.94,80.60,30.0,0.0,5.82,ZA,1589126055,Blue Bay Lodge


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(
    location,
    weights = humidity,
    dissipating=False,
    max_intensity=100,
    point_radius=4)
fig.add_layer(heat_layer)
marker_loc = locations
markers = gmaps.marker_layer(marker_loc,
    info_box_template)
fig.add_layer(markers)


fig

# Display figure


Figure(layout=FigureLayout(height='420px'))